# Chapter 2: Working with Text Data

- Learning how to prepare and process text data for language modeling
- Understanding tokenization and vocabulary creation
- Converting raw text into a format suitable for model training

Packages that are being used in this notebook:

In [ ]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

## 2.1 Understanding word embeddings

- Embeddings can represent different types of data, but for LLMs we primarily work with text embeddings
- Language models operate in high-dimensional vector spaces (typically hundreds or thousands of dimensions)
- Visualizing these spaces is challenging since we naturally think in 1-3 dimensions, so simplified 2D representations help build intuition

## 2.2 Tokenizing text

- Tokenization involves splitting text into discrete units (words, punctuation, subwords)
- These tokens become the basic building blocks that our model will process
- We'll use a public domain text: [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) as our sample data

In [ ]:
import os

# Check for local file first (zero-copy policy compliance)
# The file should already exist in the notebook directory
file_path = "the-verdict.txt"

if not os.path.exists(file_path):
    # Fallback: download from public domain source if local file missing
    # Note: This is a public domain text by Edith Wharton
    import requests
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded {file_path} from external source")
else:
    print(f"Using local file: {file_path}")


In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

- Our objective: convert the raw text into tokens that can be processed by a language model
- We'll start by building a basic tokenizer using simple example text, then apply it to our full dataset
- First, let's see how a basic regex pattern can split text on whitespace characters

In [ ]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

- Splitting only on whitespace isn't sufficient - we also need to handle punctuation like commas and periods
- Let's enhance our regex pattern to capture these punctuation marks as separate tokens

In [ ]:
result = re.split(r'([,.]|\s)', text)

print(result)

- The current approach produces empty strings in our token list
- We need to filter these out to get clean, meaningful tokens

In [ ]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

- The tokenizer is improving, but we should expand it to handle more punctuation types
- Let's add support for question marks, exclamation marks, and other common punctuation

In [ ]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

- Our tokenizer pattern looks solid now
- Time to apply it to the full text we loaded earlier

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

- Now let's count how many tokens we've extracted from the text

In [ ]:
print(len(preprocessed))

## 2.3 Converting tokens into IDs

- To work with tokens computationally, we need to map them to numeric IDs
- We'll build a vocabulary dictionary that assigns each unique token a unique integer identifier
- First step: extract all unique tokens and organize them alphabetically

In [ ]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)

print(vocab_size)

- Now we'll construct the vocabulary mapping: each token gets assigned a sequential integer ID

In [ ]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- Let's inspect the first 50 token-to-ID mappings to see how our vocabulary looks

In [ ]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

- Below, we illustrate the tokenization of a short sample text using a small vocabulary:
- Putting it now all together into a tokenizer class

In [36]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- The `encode` function turns text into token IDs
- The `decode` function turns token IDs back into text
- We can use the tokenizer to encode (that is, tokenize) texts into integers
- These integers can then be embedded (later) as input of/for the LLM

In [37]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- We can decode the integers back into text


In [38]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [39]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'